# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


In [2]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
df = pd.read_csv("../output_data/cities.csv")
df = df.drop(["Unnamed: 0"], axis=1)
df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Carnarvon,0,AU,1586138776,78,-24.87,113.63,298.15,5.70
1,Arraial do Cabo,77,BR,1586138779,69,-22.97,-42.03,297.28,3.58
2,Bilibino,42,RU,1586138780,82,68.05,166.44,256.73,1.06
3,Provideniya,100,RU,1586138781,66,64.38,-173.30,262.15,7.00
4,Hilo,90,US,1586138642,65,19.73,-155.09,299.15,5.70
...,...,...,...,...,...,...,...,...,...
566,Kiryat Gat,40,IL,1586139436,87,31.61,34.76,289.15,2.10
567,Arrondissement de Saint-Denis,0,FR,1586139437,53,48.92,2.33,286.15,3.70
568,Ballina,10,AU,1586139233,60,-28.87,153.57,298.71,4.10
569,Cimarron Hills,20,US,1586139439,20,38.86,-104.70,290.15,5.10


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Store latitude and longitude in locations
locations = df[["Lat", "Lng"]]

# Fill NaN values and convert to float
rating = df["Humidity"].astype(float)

# Plot Heatmap
humid_fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add layer
humid_fig.add_layer(heat_layer)

# Display figure
humid_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Max temperature - 65F to 75F (292K to 297K)
* Humidity - 30mmHg to 60mmHg
* Wind Speed - 1-4mph

* Drop any rows will null values.

In [5]:
# Cities with ideal temperature
temp_df = df.loc[(df["Max Temp"] >= 292) & (df["Max Temp"] <= 297)]
temp_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
5,Albuquerque,40,US,1586138255,13,35.08,-106.65,293.71,7.70
35,Kabalo,100,CD,1586138816,79,-6.05,26.92,295.22,0.36
36,East London,65,ZA,1586138801,75,-33.02,27.91,292.59,6.15
42,Mar del Plata,0,AR,1586138532,82,-38.00,-57.56,293.15,6.09
46,Presidencia Roque Sáenz Peña,100,AR,1586138829,91,-26.79,-60.44,292.48,2.24
...,...,...,...,...,...,...,...,...,...
537,Paramirim,99,BR,1586139402,68,-13.44,-42.24,296.50,2.57
549,Kisangani,100,CD,1586139415,86,0.52,25.20,293.93,0.40
550,Pontes e Lacerda,96,BR,1586139417,86,-15.23,-59.34,296.74,0.99
554,Bongandanga,100,CD,1586139421,84,1.50,21.05,294.43,0.88


In [6]:
# Cities with ideal humidity
humid_df = temp_df.loc[(temp_df["Humidity"] >= 30) & (temp_df["Humidity"] <= 60)]
humid_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
58,Ejido,100,VE,1586138844,51,8.55,-71.24,294.83,0.86
229,Dalby,0,AU,1586139044,35,-27.18,151.27,296.15,3.10
298,Kiama,75,AU,1586139122,56,-34.68,150.87,294.26,4.60
328,Itoman,75,JP,1586139156,56,26.12,127.67,294.15,3.60
369,Ulladulla,75,AU,1586139202,56,-35.35,150.47,293.15,4.60
393,Nishihara,75,JP,1586139229,60,26.18,127.76,295.15,3.60
425,Airai,12,TL,1586139267,46,-8.93,125.41,296.92,1.28
432,Beloha,72,MG,1586139275,59,-25.17,45.05,296.21,3.07
505,Alice Springs,81,AU,1586139346,31,-23.70,133.88,296.15,5.70
534,Renqiu,28,CN,1586139399,35,38.70,116.09,293.25,1.19


In [7]:
# Cities with ideal wind speed
ideal_df = humid_df.loc[(humid_df["Wind Speed"] >= 1) & (humid_df["Wind Speed"] <= 4)]
ideal_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
229,Dalby,0,AU,1586139044,35,-27.18,151.27,296.15,3.10
328,Itoman,75,JP,1586139156,56,26.12,127.67,294.15,3.60
393,Nishihara,75,JP,1586139229,60,26.18,127.76,295.15,3.60
425,Airai,12,TL,1586139267,46,-8.93,125.41,296.92,1.28
432,Beloha,72,MG,1586139275,59,-25.17,45.05,296.21,3.07
534,Renqiu,28,CN,1586139399,35,38.70,116.09,293.25,1.19


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = ideal_df
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
229,Dalby,0,AU,1586139044,35,-27.18,151.27,296.15,3.10
328,Itoman,75,JP,1586139156,56,26.12,127.67,294.15,3.60
393,Nishihara,75,JP,1586139229,60,26.18,127.76,295.15,3.60
425,Airai,12,TL,1586139267,46,-8.93,125.41,296.92,1.28
432,Beloha,72,MG,1586139275,59,-25.17,45.05,296.21,3.07
534,Renqiu,28,CN,1586139399,35,38.70,116.09,293.25,1.19


In [9]:
# Search for nearest hotels
# coordinates = hotel_df[["Lat","Lng"]]
target_type = "lodging"
target_search = "hotel"

# set up a parameters dictionary
params = {
    "rankby": "distance",
    "types": target_type,
    "keyword": target_search,
    "key": g_key
        }

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    coordinates = f"{row['Lat']},{row['Lng']}"
    params['location'] = coordinates
    response = requests.get(base_url, params=params).json()
    results = response['results']
#     print(json.dumps(results[0]["name"], indent =4, sort_keys=True))
    
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        print(f"Appending data for {row['City']}")
    except:
        print(f"Missing field/result for {row['City']}... skipping.")
        
print("Done processing...")

C:\Users\rylac\Anaconda3\envs\PythonData2020_1\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\rylac\Anaconda3\envs\PythonData2020_1\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Appending data for Dalby
Appending data for Itoman
Appending data for Nishihara
Appending data for Airai
Missing field/result for Beloha... skipping.
Appending data for Renqiu
Done processing...


In [10]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
229,Dalby,0,AU,1586139044,35,-27.18,151.27,296.15,3.10,Dalby Apartments Self Contained Motel Accommod...
328,Itoman,75,JP,1586139156,56,26.12,127.67,294.15,3.60,Southern Beach Hotel & Resort Okinawa
393,Nishihara,75,JP,1586139229,60,26.18,127.76,295.15,3.60,かりゆしコンドミニアムリゾート与那原 ＳＵＮＲＩＺＥ・ＯＣＥＡＮ
425,Airai,12,TL,1586139267,46,-8.93,125.41,296.92,1.28,Restaurant Gabmenis
432,Beloha,72,MG,1586139275,59,-25.17,45.05,296.21,3.07,NaN
534,Renqiu,28,CN,1586139399,35,38.70,116.09,293.25,1.19,Haisheng International Hotel


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>

"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
hover_layer = "Click to know more"

In [22]:
# Add marker layer ontop of heat map

humid_fig
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info, hover_text=hover_layer)
humid_fig.add_layer(marker_layer)

# Display Map
humid_fig


Figure(layout=FigureLayout(height='420px'))